# Chapter 6: Object References, Mutability, and Recycling

## Variables are not boxes

Example 6-1: Variables a and b hold reference to the same list, not copies of the list

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

Example 6-2. Variables are bound to objects only after the objects are created

In [2]:
class Gizmo:
    def __init__(self):
        print(f"Gizmo id: {id(self)}")

x = Gizmo()

Gizmo id: 140436782964448


In [3]:
# This will raise an error. Will this gizmo be instantiated?
fantastic = Gizmo() * 10

Gizmo id: 140436783620256


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [4]:
fantastic

NameError: name 'fantastic' is not defined

In [5]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'quit',
 'x']

In [6]:
# variable y was never created. Why?

## Identity, Equality, and Aliases

Example 6-3: charles and lewis refer to the same object

In [7]:
charles = {
    "name": "Charles L. Dodgson",
    "born": 1832,
}

lewis = charles

lewis is charles

True

In [8]:
id(lewis), id(charles)

(140436815107648, 140436815107648)

In [9]:
lewis["balance"] = 950

In [10]:
# will charles be changed too?
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

Example 6-4. alex and charles compare equal, but alex is not charles

In [11]:
alex = {
    "name": "Charles L. Dodgson",
    "born": 1832,
    "balance": 950,
    "m": {1:2},
}
alex == charles

False

In [12]:
alex is charles

False

In [13]:
lewis["m"] = {1:2}

Example 6-4. alex and charles

## Choosing between == and `is`

In [14]:
x = None

x is None

True

In [15]:
x is not None

False

In [16]:
x == None

True

In [17]:
# which is faster, == or is? Why?

## The Relative Immutability of Tuples

In [18]:
# Tuples are containers. They hold references to objects.

Example 6-5. `t1` and `t2` initially compare equal, but chagning a mutable item inside tuple `t1` makes it different

In [19]:
t1 = (1, 2, [30, 40])

In [20]:
t2 = (1, 2, [30, 40])

In [21]:
t1 == t2

True

In [22]:
id(t1[-1])

140436815394560

In [23]:
t1[-1].append(99)

In [24]:
t1

(1, 2, [30, 40, 99])

In [25]:
id(t1[-1])

140436815394560

In [26]:
t1 == t2

False

## Copies are Shallow by Default

In [27]:
l1 = [3, [55, 44], (7, 8, 9)]

In [28]:
l2 = list(l1)

In [29]:
l2

[3, [55, 44], (7, 8, 9)]

In [30]:
l2 == l1

True

In [31]:
l2 is l1

False

Example 6-6. Making a shallow copy of a list containing another list

In [32]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [66, 55, 44], (7, 8, 9)]

In [33]:
l1.append(100)

In [34]:
l1[1].remove(55)

In [35]:
l1

[3, [66, 44], (7, 8, 9), 100]

In [36]:
print(f"l1: {l1}")
print(f"l2: {l2}")

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]


In [37]:
l2[1] += [33, 22]
l2

[3, [66, 44, 33, 22], (7, 8, 9)]

In [38]:
l2[2] = l2[2] + (10, 11)

In [39]:
print(f"l1: {l1}")
print(f"l2: {l2}")

l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


In [40]:
l1[1] is l2[1]

True

## Deep and Shallow Copies of Arbitrary Objects

Example 6-8. Bus picks up and drops off passengers

In [42]:


class Bus:

    def __init__(self, passengers: Optional[list] = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


Example 6-9. Effects of using `copy` vs `deepcopy`

In [43]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

In [44]:
bus1.drop('Bill')

In [45]:
bus2.passengers

['Alice', 'Claire', 'David']

In [46]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

Example 6-10. Cyclic references: `b` refers to `a`, and then is appended to `a`; `deepcopy` still manages to copy `a`

In [47]:
a = [10, 20]

In [48]:
b = [a, 30]

In [49]:
a.append(b)

In [56]:
a[2][0][2][0][2][0][2]

[[10, 20, [...]], 30]

In [58]:
from copy import deepcopy

c = deepcopy(a)
c[2][0]

[10, 20, [[...], 30]]

## Function Parameters as References

Example 6-11. A function may change any mutable object it receives

In [65]:
def f(a, b):
    a += b
    return a

x = 1
y = 2
f(x, y)

3

In [66]:
x, y

(1, 2)

In [67]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [68]:
a, b

([1, 2], [3, 4])

In [69]:
t = (10, 20)
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [70]:
t, u

((10, 20), (30, 40))

## Mutable Types as Parameter Defaults: Bad Idea

Example 6-12. A simple class to illustrate the danger of a mutable default

In [72]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):  # <1>
        self.passengers = passengers  # <2>

    def pick(self, name):
        self.passengers.append(name)  # <3>

    def drop(self, name):
        self.passengers.remove(name)

In [73]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [74]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [75]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [76]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [77]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [78]:
bus2.passengers is bus3.passengers

True

In [79]:
bus1.passengers

['Bill', 'Charlie']

In [80]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [81]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [82]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

## Defensive Programming with Mutable Parameters

Example 6-14. Passengers disappear when dropped by a `TwilightBus`

In [89]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  # <1>
        else:
            self.passengers = list(passengers)  #<2>

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)  # <3>



In [90]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)

In [91]:
bus.drop('Tina')
bus.drop('Pat')
bus.passengers

['Sue', 'Maya', 'Diana']

In [92]:
basketball_team

['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

## `del` and Garbage Collection

In [93]:
a = [1, 2]
b = a
del a
b

[1, 2]

In [94]:
b = [3]

Example 6-16. Watching the end of an object when no more references point to it

In [95]:
import weakref
s1 = {1, 2, 3}
s2 = s1

def bye():
    print("...like tears in the rain.")

In [96]:
# register the bye callback on the object referred by s1
ender = weakref.finalize(s1, bye)
ender.alive

True

In [97]:
del s1

In [98]:
ender.alive

True

In [99]:
s2 = "spam"

...like tears in the rain.


In [100]:
ender.alive

False

`del` does not delete objects, but objects may be deleted as a consequence of being unreachable after `del` is used.

## Tricks Python Plays with Immutables

Example 6-17. A tuple built from another is actually the same exact tuple

In [101]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t1 is t2

True

In [102]:
t3 = t1[:]

In [103]:
t3 is t1

True

In [ ]:
# The same behavior can be observed with instances of str, bytes, and frozenset.

Example 6-18. String literals may create shared objects

In [104]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

False

In [106]:
s1 = "ABC"
s2 = "ABC"
s2 is s1

s2 = ""
s2 is s1

False

In [ ]:
# this is an optimization technique called interning.